# ハイパーパラメータのチューニング

## Data Load

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mlp
import seaborn as sns
import numpy as np

pd.set_option('display.max_columns', 200)
plt.style.use('ggplot')

電気通信事業者の解約データを読み込む  
(https://www.kaggle.com/blastchar/telco-customer-churn)

In [2]:
input_path = '../data'
df = pd.read_csv(os.path.join(input_path, 'WA_Fn-UseC_-Telco-Customer-Churn.csv'))

# TotalCharges列に空文字が存在して文字列型になっているので欠損値に置換して少数型にしておく
col = 'TotalCharges'
df[col] = df[col].replace({' ': np.nan}).astype(float)

df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


## Data Partition

In [3]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, train_size=0.8, random_state=2021, shuffle=True)
print('original_size:', df.shape)
print('train_size:', train_df.shape)
print('test_size:', test_df.shape)

original_size: (7043, 21)
train_size: (5634, 21)
test_size: (1409, 21)


## Data Preparation

In [4]:
from sklearn.preprocessing import StandardScaler
from category_encoders import OrdinalEncoder

In [5]:
# ターゲットを変換
train_df['Churn'] = train_df['Churn'].map({'Yes':1, 'No':0})
test_df['Churn'] = test_df['Churn'].map({'Yes':1, 'No':0})

# 数値変数の標準化
# 今回はツリー系アルゴリズムを使用する本来は不要だが、勉強のために実施しておく
num_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
scaler = StandardScaler()
train_df[num_cols] = scaler.fit_transform(train_df[num_cols])
test_df[num_cols] = scaler.transform(test_df[num_cols])

# customerID以外のカテゴリ変数をOrdinalエンコーディング
cat_cols = []
for col in train_df.columns:
    if train_df[col].dtype == 'object':
        cat_cols.append(col)
cat_cols.remove('customerID')
encoder = OrdinalEncoder()
train_df[cat_cols] = encoder.fit_transform(train_df[cat_cols])
test_df[cat_cols] = encoder.transform(test_df[cat_cols])

# 欠損値をトレーニングデータの中央値で保管
train_df.fillna(train_df.median(), inplace=True)
test_df.fillna(train_df.median(), inplace=True)

# 前処理後のデータプレビュー
train_df.head()

/var/folders/v6/1f2j3j154zj9g03clj2_yt_00000gn/T/ipykernel_10024/1495117370.py:23: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train_df.fillna(train_df.median(), inplace=True)
/var/folders/v6/1f2j3j154zj9g03clj2_yt_00000gn/T/ipykernel_10024/1495117370.py:24: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  test_df.fillna(train_df.median(), inplace=True)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
6125,0871-URUWO,1,0,1,1,-0.791990,1,1,1,1,1,1,1,1,1,1,1,1,1.255598,-0.403651,1
6958,3078-ZKNTS,2,0,1,2,-0.791990,1,2,2,2,2,2,2,2,2,2,1,2,-1.485131,-0.894834,0
4062,1915-IOFGU,2,0,2,1,-1.280574,1,2,1,1,1,3,1,3,3,1,2,3,0.200833,-0.972643,1
5298,5647-FXOTP,2,1,1,1,1.121629,1,1,1,1,3,1,1,1,1,1,1,3,1.376855,1.822967,0
1214,9866-QEVEE,1,0,2,1,-0.547698,1,1,1,1,1,3,1,3,1,1,1,2,0.715758,-0.327057,1


In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5634 entries, 6125 to 1140
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        5634 non-null   object 
 1   gender            5634 non-null   int64  
 2   SeniorCitizen     5634 non-null   int64  
 3   Partner           5634 non-null   int64  
 4   Dependents        5634 non-null   int64  
 5   tenure            5634 non-null   float64
 6   PhoneService      5634 non-null   int64  
 7   MultipleLines     5634 non-null   int64  
 8   InternetService   5634 non-null   int64  
 9   OnlineSecurity    5634 non-null   int64  
 10  OnlineBackup      5634 non-null   int64  
 11  DeviceProtection  5634 non-null   int64  
 12  TechSupport       5634 non-null   int64  
 13  StreamingTV       5634 non-null   int64  
 14  StreamingMovies   5634 non-null   int64  
 15  Contract          5634 non-null   int64  
 16  PaperlessBilling  5634 non-null   int64

## Modeling

LightGBMでモデルを作成する  
バリデーションスキームはLeave One Outで、ランダムに8:2に分割する

In [7]:
import lightgbm as lgb

In [ ]:
# ターゲットと特徴量に分離
train_x = train_df.drop(['customerID','Churn'], axis=1)
train_y = train_df['Churn']
test_x = test_df.drop(['customerID','Churn'], axis=1)
test_y = test_df['Churn']

# EarlyStopping用にさらにデータを分割する
train_x_, val_x, train_y_, val_y = train_test_split(train_x, train_y, train_size=0.8, random_state=888, shuffle=True)

# 学習用のデータ
dtrain = lgb.Dataset(data=train_x_, label=train_y_)
# EarlyStoppingのためのデータ
dval = lgb.Dataset(data=val_x, label=val_y, reference=dtrain)

# ハイパーパラメータ設定
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'learning_rate': 0.02,
    'num_leaves': 15,
    'bagging_freq': 5,
    'bagging_fraction': 0.8,
    'feature_fraction': 0.8,
    'random_state': 777,
    'min_data_in_leaf': 1,
    'lambda_l1': 0,
    'lambda_l2': 0,
    'verbose': -1
}

# モデルの学習実行
model = lgb.train(
    params, dtrain, num_boost_round=9999, valid_sets=dval, early_stopping_rounds=200
)

In [47]:
# バリデーションスコア
from sklearn.metrics import roc_auc_score

val_pred = model.predict(val_x)
print('validation_score(AUC):', roc_auc_score(y_true=val_y, y_score=val_pred))

validation_score(AUC): 0.8311496770480271


## ハイパーパラメータチューニング前のテストスコア

In [45]:
# 推論実行
test_pred = model.predict(test_x)

print('test_score(AUC):', roc_auc_score(y_true=test_y, y_score=test_pred))

test_score(AUC): 0.848412674294339


In [48]:
# 結果を格納しておく
results = {}

result = {
    'model': 'LightGBM',
    'validation_score(AUC)': roc_auc_score(y_true=val_y, y_score=val_pred),
    'test_score(AUC)': roc_auc_score(y_true=test_y, y_score=test_pred)
}
results['ManuallySelected'] = result

results

{'ManuallySelected': {'model': 'LightGBM',
  'validation_score(AUC)': 0.8311496770480271,
  'test_score(AUC)': 0.848412674294339}}

In [20]:
# テスト用のデータを出力しておく
test_x.to_csv(os.path.join(input_path, 'test_x.csv'), encoding='utf-8', index=False)
test_y.to_csv(os.path.join(input_path, 'test_y.csv'), encoding='utf-8', index=False)

## GridSearch

GridSearchでLightGBMのハイパーパラメータをチューニングする  
チューニング対象は以下の通り  
- num_leaves
- bagging_fraction
- feature_fraction
- lambda_l1
- lambda_l2

In [14]:
from sklearn.model_selection import GridSearchCV

In [69]:
# GridSearchで探索するハイパーパラメータ空間を定義
search_params = {
    'num_leaves': [7, 15],
    'subsample': [0.7, 0.9], # bagging_fractionのエイリアス
    'colsample_bytree': [0.7, 0.9], # feature_fractionのエイリアス
    'reg_alpha': [0, 0.1], # lambda_l1のエイリアス
    'reg_lambda': [0, 0.1] # lambda_l2のエイリアス
}

# scikit-learnを使ってGridSearchを実施するため、LightGBMのオリジナルAPIではなく、sklearnのラッパーAPIを使用する
clf = lgb.LGBMClassifier(max_depth=-1, random_state=777, n_estimators=2000, verbose=0,
                        boosting_type='gbdt', importance_type='gain', learning_rate=0.02,
                        min_child_samples=1, force_col_wise=True, subsample_freq=5)

gs = GridSearchCV(estimator=clf, param_grid=search_params, scoring='neg_log_loss', cv=3, verbose=0)

gs.fit(train_x_, train_y_)

GridSearchCV(cv=3,
             estimator=LGBMClassifier(force_col_wise=True,
                                      importance_type='gain',
                                      learning_rate=0.02, min_child_samples=1,
                                      n_estimators=2000, random_state=777,
                                      subsample_freq=5, verbose=0),
             param_grid={'colsample_bytree': [0.7, 0.9], 'num_leaves': [7, 15],
                         'reg_alpha': [0, 0.1], 'reg_lambda': [0, 0.1],
                         'subsample': [0.7, 0.9]},
             scoring='neg_log_loss')

In [114]:
# GridSearchで探索した最良のハイパーパラメータの組み合わせ
best_params = gs.best_params_
best_params

{'colsample_bytree': 0.7,
 'num_leaves': 7,
 'reg_alpha': 0.1,
 'reg_lambda': 0.1,
 'subsample': 0.9}

In [ ]:
# GridSearchの結果を使用してモデルをトレーニング
gs_model = lgb.LGBMClassifier(max_depth=-1, random_state=777, n_estimators=9999, verbose=0,
                        boosting_type='gbdt', importance_type='gain', learning_rate=0.02,
                        min_child_samples=1, force_col_wise=True, subsample_freq=5, **best_params)

gs_model.fit(train_x_, train_y_, early_stopping_rounds=200, eval_metric='logloss', eval_set=[(val_x, val_y)])

In [116]:
# バリデーションスコア
from sklearn.metrics import roc_auc_score

val_pred_gs = gs_model.predict(val_x)
print('validation_score(AUC):', roc_auc_score(y_true=val_y, y_score=val_pred_gs))

validation_score(AUC): 0.6989671931956257


In [117]:
# 推論実行
test_pred_gs = gs_model.predict(test_x)

print('test_score(AUC):', roc_auc_score(y_true=test_y, y_score=test_pred_gs))

test_score(AUC): 0.7127581176971116


In [89]:
# 結果を格納しておく

result = {
    'model': 'LightGBM',
    'validation_score(AUC)': roc_auc_score(y_true=val_y, y_score=val_pred_gs),
    'test_score(AUC)': roc_auc_score(y_true=test_y, y_score=test_pred_gs)
}
results['GridSearch'] = result

results

{'ManuallySelected': {'model': 'LightGBM',
  'validation_score(AUC)': 0.8311496770480271,
  'test_score(AUC)': 0.848412674294339},
 'GridSearch': {'model': 'LightGBM',
  'validation_score(AUC)': 0.6989671931956257,
  'test_score(AUC)': 0.7127581176971116}}

## RandomSearch

RandomSearchでLightGBMのハイパーパラメータをチューニングする  
チューニング対象は以下の通り  
- num_leaves
- bagging_fraction
- feature_fraction
- lambda_l1
- lambda_l2

In [77]:
from sklearn.model_selection import RandomizedSearchCV
import scipy as sp

In [ ]:
# RandomSearchで探索するハイパーパラメータ空間を定義
search_params = {
    'num_leaves': sp.stats.randint(6, 15),
    'subsample': sp.stats.uniform(loc=0.4, scale=0.5), # bagging_fractionのエイリアス
    'colsample_bytree': sp.stats.uniform(loc=0.4, scale=0.5), # feature_fractionのエイリアス
    'reg_alpha': sp.stats.uniform(loc=0, scale=1), # lambda_l1のエイリアス
    'reg_lambda': sp.stats.uniform(loc=0, scale=1) # lambda_l2のエイリアス
}

# scikit-learnを使ってRandomSearchを実施するため、LightGBMのオリジナルAPIではなく、sklearnのラッパーAPIを使用する
clf = lgb.LGBMClassifier(max_depth=-1, random_state=777, n_estimators=2000, verbose=0,
                        boosting_type='gbdt', importance_type='gain', learning_rate=0.02,
                        min_child_samples=1, force_col_wise=True, subsample_freq=5)

rs = RandomizedSearchCV(estimator=clf, param_distributions=search_params, n_iter=32, scoring='neg_log_loss', cv=3, verbose=0)

rs.fit(train_x, train_y)

In [93]:
# RandomSearchで探索した最良のハイパーパラメータの組み合わせ
best_params = rs.best_params_
best_params

{'colsample_bytree': 0.5174751806969153,
 'num_leaves': 7,
 'reg_alpha': 0.920198479937528,
 'reg_lambda': 0.4169366515580416,
 'subsample': 0.7994821470272152}

In [ ]:
# RandomSearchの結果を使用してモデルをトレーニング
rs_model = lgb.LGBMClassifier(max_depth=-1, random_state=777, n_estimators=9999, verbose=0,
                        boosting_type='gbdt', importance_type='gain', learning_rate=0.02,
                        min_child_samples=1, force_col_wise=True, subsample_freq=5, **best_params)

rs_model.fit(train_x_, train_y_, early_stopping_rounds=200, eval_metric='logloss', eval_set=[(val_x, val_y)])

In [95]:
# バリデーションスコア
from sklearn.metrics import roc_auc_score

val_pred_rs = rs_model.predict(val_x)
print('validation_score(AUC):', roc_auc_score(y_true=val_y, y_score=val_pred_rs))

validation_score(AUC): 0.7014712700645903


In [96]:
# 推論実行
test_pred_rs = rs_model.predict(test_x)

print('test_score(AUC):', roc_auc_score(y_true=test_y, y_score=test_pred_rs))

test_score(AUC): 0.7104852638029974


In [97]:
# 結果を格納しておく

result = {
    'model': 'LightGBM',
    'validation_score(AUC)': roc_auc_score(y_true=val_y, y_score=val_pred_rs),
    'test_score(AUC)': roc_auc_score(y_true=test_y, y_score=test_pred_rs)
}
results['RandomSearch'] = result

results

{'ManuallySelected': {'model': 'LightGBM',
  'validation_score(AUC)': 0.8311496770480271,
  'test_score(AUC)': 0.848412674294339},
 'GridSearch': {'model': 'LightGBM',
  'validation_score(AUC)': 0.6989671931956257,
  'test_score(AUC)': 0.7127581176971116},
 'RandomSearch': {'model': 'LightGBM',
  'validation_score(AUC)': 0.7014712700645903,
  'test_score(AUC)': 0.7104852638029974}}

## Optuna

OptunaでLightGBMのハイパーパラメータをチューニングする  
チューニング対象は以下の通り  
- num_leaves
- bagging_fraction
- feature_fraction
- lambda_l1
- lambda_l2

In [98]:
import optuna

In [99]:
# 目的関数の定義
def objective(trial):
    train_x_, val_x, train_y_, val_y = train_test_split(train_x, train_y, train_size=0.8, random_state=888, shuffle=True)

    dtrain = lgb.Dataset(data=train_x_, label=train_y_)
    dval = lgb.Dataset(data=val_x, label=val_y, reference=dtrain)

    params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting_type': 'gbdt',
        'max_depth': -1,
        'learning_rate': 0.02,
        'num_leaves': trial.suggest_int('num_leaves',6, 15),
        'bagging_freq': 5,
        'bagging_fraction': trial.suggest_loguniform('bagging_fraction', 0.4, 1.0),
        'feature_fraction': trial.suggest_loguniform('feature_fraction', 0.4, 1.0),
        'random_state': 777,
        'min_data_in_leaf': 1,
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-3, 1.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-3, 1.0),
        'verbose': -1
    }

    model = lgb.train(
        params, dtrain, num_boost_round=9999, valid_sets=dval, early_stopping_rounds=200
    )
    
    val_pred = model.predict(val_x)
    score = roc_auc_score(y_true=val_y, y_score=val_pred)
    
    return score

In [ ]:
# 最適化処理の実行
study = optuna.create_study()
study.optimize(func=objective, n_trials=32, timeout=None, n_jobs=-1)

In [101]:
# Optunaで探索した最良のハイパーパラメータの組み合わせ
best_params = study.best_params
best_params

{'num_leaves': 6,
 'bagging_fraction': 0.9759474929453683,
 'feature_fraction': 0.5382475486628867,
 'lambda_l1': 0.08115986968807415,
 'lambda_l2': 0.05554908434047863}

In [104]:
# Optunaの探索結果を使って再トレーニング
params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'max_depth': -1,
    'learning_rate': 0.02,
    #'num_leaves': trial.suggest_int('num_leaves',6, 15),
    'bagging_freq': 5,
    #'bagging_fraction': trial.suggest_loguniform('bagging_fraction', 0.4, 1.0),
    #'feature_fraction': trial.suggest_loguniform('feature_fraction', 0.4, 1.0),
    'random_state': 777,
    'min_data_in_leaf': 1,
    #'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-3, 1.0),
    #'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-3, 1.0),
    'verbose': -1
}
params.update(best_params)

# モデルの学習実行
model = lgb.train(
    params, dtrain, num_boost_round=9999, valid_sets=dval, early_stopping_rounds=200
)

/Users/suzuki001/Desktop/studyML/handson/src/venv/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	valid_0's binary_logloss: 0.57897
Training until validation scores don't improve for 200 rounds
[2]	valid_0's binary_logloss: 0.574309
[3]	valid_0's binary_logloss: 0.571158
[4]	valid_0's binary_logloss: 0.567626
[5]	valid_0's binary_logloss: 0.564067
[6]	valid_0's binary_logloss: 0.560381
[7]	valid_0's binary_logloss: 0.556767
[8]	valid_0's binary_logloss: 0.553553
[9]	valid_0's binary_logloss: 0.551362
[10]	valid_0's binary_logloss: 0.548292
[11]	valid_0's binary_logloss: 0.545012
[12]	valid_0's binary_logloss: 0.541722
[13]	valid_0's binary_logloss: 0.539303
[14]	valid_0's binary_logloss: 0.536958
[15]	valid_0's binary_logloss: 0.534073
[16]	valid_0's binary_logloss: 0.532057
[17]	valid_0's binary_logloss: 0.530133
[18]	valid_0's binary_logloss: 0.527535
[19]	valid_0's binary_logloss: 0.524964
[20]	valid_0's binary_logloss: 0.52249
[21]	valid_0's binary_logloss: 0.520041
[22]	valid_0's binary_logloss: 0.517791
[23]	valid_0's binary_logloss: 0.515614
[24]	valid_0's binary_logloss

In [105]:
# バリデーションスコア
from sklearn.metrics import roc_auc_score

val_pred_optuna = model.predict(val_x)
print('validation_score(AUC):', roc_auc_score(y_true=val_y, y_score=val_pred_optuna))

validation_score(AUC): 0.8272646607405513


In [106]:
# 推論実行
test_pred_optuna = model.predict(test_x)

print('test_score(AUC):', roc_auc_score(y_true=test_y, y_score=test_pred_optuna))

test_score(AUC): 0.8492888942914014


In [107]:
# 結果を格納しておく

result = {
    'model': 'LightGBM',
    'validation_score(AUC)': roc_auc_score(y_true=val_y, y_score=val_pred_optuna),
    'test_score(AUC)': roc_auc_score(y_true=test_y, y_score=test_pred_optuna)
}
results['Optuna'] = result

results

{'ManuallySelected': {'model': 'LightGBM',
  'validation_score(AUC)': 0.8311496770480271,
  'test_score(AUC)': 0.848412674294339},
 'GridSearch': {'model': 'LightGBM',
  'validation_score(AUC)': 0.6989671931956257,
  'test_score(AUC)': 0.7127581176971116},
 'RandomSearch': {'model': 'LightGBM',
  'validation_score(AUC)': 0.7014712700645903,
  'test_score(AUC)': 0.7104852638029974},
 'Optuna': {'model': 'LightGBM',
  'validation_score(AUC)': 0.8272646607405513,
  'test_score(AUC)': 0.8492888942914014}}